# Can we predict the category of an app from the app's description?

We wanted to see if we can create a Machine Learning model that will accuretly predict the category of an app from the app's description. To do this we created a web scraper to scrape data from the GooglePlay Store. After cleaning our data, we performed NLTK, Feature Engineering and Model Fitting to create an optimal ML Model. 

<br>

<br>

## Web Scraping

Here we created a web scraper to collect 60 app descriptions per category for 18 categories. 